In [7]:
query = """
with base_data as (
  select * from all_matches WHERE dt > '2016-12-03' AND dt < '2021-11-07'  AND match_type = 'T20'
), dismissals as (
  select sum("is_wicket") as dismissal_count from base_data
)
SELECT
  sum("batter_runs")*100.0/sum("delivery_count") as strike_rate,
  sum("batter_runs")*1.0/(SELECT sum(dismissal_count) from dismissals) as average
from base_data
"""

In [2]:
from typing import List
from pyspark import Row
from pyspark.sql import SparkSession, DataFrame
from etl.commons.stats_helper import map_player_match_for_training, sequential_player_match_for_training
from etl.commons.training_record_schema import training_record_schema
from etl.datawriter import write_dataframe

spark = SparkSession.builder.appName('SparkByExamples.com').config('spark.driver.bindAddress','localhost').config("spark.ui.port","4051").config("spark.driver.memory","3g").getOrCreate()

23/02/11 21:49:09 WARN Utils: Your hostname, Chaturvedi_PC resolves to a loopback address: 127.0.1.1; using 172.27.69.129 instead (on interface eth0)
23/02/11 21:49:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/11 21:49:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.read.parquet("processed_output/delivery_parquet").registerTempTable("all_matches")

23/02/11 21:49:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
spark.sql("select 1+1 as x from all_matches limit 10").collect()

[Row(x=2),
 Row(x=2),
 Row(x=2),
 Row(x=2),
 Row(x=2),
 Row(x=2),
 Row(x=2),
 Row(x=2),
 Row(x=2),
 Row(x=2)]